# The Idea
The main idea of this project is to simulate the heating of steak to find the optimal conditions for cooking it to a medium-rare rarity.

# Preamble
We import dependencies and define constants and dictionaries here.

## Importing Packages

In [ ]:
import copy
import random
import numpy as np
import matplotlib.pyplot as plt
import math

## Defining Constants

In [6]:
# fat
fat_shc = 1.9842e3 # J kg-1 K-1
fat_tc = 1.8071e-1 # W m-1 K-1
fat_density = 9.2559e2 # kg m-3

# protein
protein_shc = 2.0082e3 # J kg-1 K-1
protein_tc = 1.7881e-1 # W m-1 K-1
protein_density = 1.3299e3 # kg m-3

# water
water_shc = 4.184e3 # J kg-1 K-1
water_tc = 6e-1 # W m-1 K-1
water_density = 9.97e2 # kg m-3

# pixel
pixel_side_length = 0.0025 # meter

# temperatures
max_temperature = 220
room_temperature = 25 # celsius
heating_temperature = 200 # celsius
ideal_internal_temperature = 60 # celsius

# temperature thresholds
maillard_temp = 130 # celsius
burnt_temp = 175 # celcius

# composition
fat_percentage = .09
water_percentage = .69
protein_percentage = 1 - fat_percentage - water_percentage

## Defining Dictionaries
In the following we define the `pxl_type` dictionary that map integer numbers to each pixel type as well as a `colors_dict` dictionary that gives us easy access to rbg values of commonly used colors 

In [3]:
pxl_type = {
        "source" : 0,
        "fat" : 1,
        "protein" : 2,
        "water" : 3,
        "maillard" : 4,
        "burnt" : 5
    }

colors_dict = {
        "GOLD" : [153/255,101/255,21/255],
        "BLACK": [0,0,0],
        "RED": [255,0,0],
        "RAW_COLOR": [230,117,122], # light pink
        "INNER_COOKED_COLOR": [158,155,147], # grey
        "OUTER_COOKED_COLOR": [94,65,37],
        "FAT": [255,255,255],
        "WATER": [0,0,255]
    }

# Creating a Pixel ADT
Here we define the pixel ADT.\
**Constructor**\
`create_pixel(temperature, thermal_conductivity, specific_heat_capacity, density, pixel_type, outer)` -> pixel\
**Accessors**\
`get_temp(pixel)` -> temperature (float)\
`get_tc(pixel)` -> thermal conductivity (float)\
`get_shc(pixel)` => specific heat capacity (float)\
`get_density(pixel)` -> density (float)\
`get_type(pixel)` -> pixel_type (int)\
`get_outer(pixel)` -> outer layer source or not (int)\
**Modifiers**\
`change_temp(pixel, new_temp)` -> None\
`change_pxl_type(pixel, new_type)` -> None

In [4]:
# =================================
# Constructor
# =================================

def create_pixel(temperature, thermal_conductivity, specific_heat_capacity, density, pxl_type_str, outer):
    return [temperature,
            thermal_conductivity,
            specific_heat_capacity,
            density,
            pxl_type[pxl_type],
            outer]

# =================================
# Accessors
# =================================

def get_temp(pixel):
    return pixel[0]

def get_tc(pixel):
    return pixel[1]

def get_shc(pixel):
    return pixel[2]

def get_density(pixel):
    return pixel[3]

def get_type(pixel):
    return pixel[4]

def get_outer(pixel):
    return pixel[5]

# =================================
# Modifiers
# =================================

def change_temp(pxl, new_temp):
    pxl[0] = new_temp

def change_pxl_type(pxl,pxl_type_str):
    pxl[4] = pxl_type[pxl_type_str]

# Creating a Steak
Here we define a function `create_steak` that returns a 2-d list of pixel ADTs with `height` rows and `width` columns.

This steak will have a randomly distributed composition of water, fat, and protein based on `fat_percentage`, `water_percentage`, and `protein_percentage`.

In [9]:
def create_steak(height, width):
    res = []
    for i in range(height + 1):
        # we define the outer threshold as the 1/8 outer layers of the steak
        outer = True if i < height * 1/8 or i > height * (1 - 1/8) - 1 else False
        tmp = []
        for j in range(width):
            if i < height:
                random_float = random.random()
                # randomly assign pixel type based on composition percentages
                if random_float < fat_percentage:
                    tmp.append(create_pixel(room_temperature,fat_tc,fat_shc, fat_density, "fat", outer))
                elif fat_percentage <= random_float < fat_percentage + water_percentage:
                    tmp.append(create_pixel(room_temperature,water_tc,water_shc, water_density, "water", outer))
                else:
                    tmp.append(create_pixel(room_temperature,protein_tc,protein_shc, protein_density, "protein", outer))
            else:
                # last layer (layer height + 1) is the heat source
                tmp.append(create_pixel(heating_temperature,0,0,0,"source",False))
        res.append(tmp)
    return res

# Changing Temperatures
Here we define a function `step_pixel` that returns the final temperature of a target pixel after a timestep of `t`.

The function first finds the heat gain of the target pixel $Q_{\text{gain}}$ using the following equation:
$$Q_{\text{gain}}=\sum_{i}\left(k_{i,*}\cdot A\cdot\frac{\Delta T_{i,*}}{\Delta x}\cdot t\right)$$
Where
* $k_{i,*}$ is the average thermal conductivity constant between pixel `i` and the pixel of interest,
* $A$ is the surface area of contact between pixels, modelled here as $(\Delta x)^2$,
* $\Delta T_{i,*}$ is $T_i - T_*$ where $T_i$ and $T_*$ is the temperature of the surrounding pixels and the pixel of interest at $t_0$ respectively, and
* $\Delta x$ is the pixel side length

It then finds the change in temperature $\Delta T$ using the following equation:
$$\Delta T = \frac{Q_{\text{gain}}-Q_{\text{surr}}}{\rho\cdot V\cdot c}$$
Where
* $Q_{\text{gain}}$ is the heat gain of the pixel of interest,
* $\rho$ is the density of the pixel of interest,
* $V$ is the volume of pixel of interest, modelled here as $(\Delta x)^3$ and
* $c$ is the specific heat capacity of the pixel of interest

$Q_{\text{surr}}$ is the heat loss to the surroundings, modelled as
$$Q_{\text{surr}} = n\cdot k_*\cdot A\cdot \frac{\Delta T_{*,\text{surr}}}{\Delta x}$$
Where
* $n$ is the number of sides of pixel exposed to the surroundings,
* $k$ is the thermal conductivity constant of the pixel of interest and
* $T_{*,\text{surr}}$ is the difference of temperature between the pixel of interest and room temperature at $t_0$

Finally, the final temperature of the pixel of interest, $T_{\text{final}}$ is
$$T_{\text{final}} = T_i + \Delta T$$
  
We also define a function `step`, which applies `step_pixel` to all pixels in a steak.

In [10]:
def step_pixel(image, i, j, t):

    n_rows = np.shape(image)[0]
    n_cols = np.shape(image)[1]

    # ===========================================
    # PART 1: GET THE SURROUNDING PIXEL DATA
    # ===========================================
    top = image[i-1,j] if i > 0 else None
    bottom = image[i+1,j] if i < n_rows-1 else None
    left = image[i,j-1] if j > 0 else None
    right = image[i,j+1] if j < n_cols-1 else None

    pixels_in_contact = np.array([x for x in [top,left,right,bottom] if x is not None])
    pixel_of_interest = image[i,j]

    # ===========================================
    # PART 2: CALCULATE HEAT GAIN
    # ===========================================
    temps = np.array(list(get_temp(pixel) for pixel in pixels_in_contact))
    center = get_temp(pixel_of_interest)
    ini_temp_diff = temps - center

    avg_tc_arr = np.zeros(len(pixels_in_contact))
    for idx, pxl in enumerate(pixels_in_contact):
        if get_type(pxl) != pxl_type["source"]:
            avg_tc_arr[idx] = (get_tc(pxl) + get_tc(pixel_of_interest))/2
        else:
            avg_tc_arr[idx] = get_tc(pixel_of_interest)
    
    heat_gain = np.sum(t * avg_tc_arr * pixel_side_length * ini_temp_diff)

    if i == 0 or j == 0 or j == n_cols-1:
        # accounting lost of heat to surroundings
        heat_loss_surr = ((4 - len(pixels_in_contact)) * 
                          pixel_side_length * 
                          get_tc(pixel_of_interest) * 
                          (room_temperature - (get_temp(pixel_of_interest))) 
                          * t)
        heat_gain -= heat_loss_surr
    
    # ===========================================
    # PART 3: CALCULATE FINAL TEMPERATURE
    # ===========================================
    vol = pixel_side_length ** 3
    rho = get_density(pixel_of_interest)
    shc = get_shc(pixel_of_interest)
    denom = vol * rho * shc
    change_in_temp = heat_gain / denom

    final_temp = get_temp(pixel_of_interest) + change_in_temp

    return final_temp.item()


def step(image, timestep):
    height = len(image)
    width = len(image[0])

    # create a copy so that the kernel refers to old temp values
    image_copy_np = np.array(copy.deepcopy(image))

    for i in range(height):
        for j in range(width):
            pxl = image[i][j]
            if get_type(pxl) != pxl_type["source"]:
                step_temperature = step_pixel(image_copy_np, i, j, timestep)
                change_temp(pxl, step_temperature)
                if get_temp(pxl) >= burnt_temp:
                    #set state to burnt
                    set_pxl_type(pxl,"burnt")
                if get_temp(pxl) >= maillard_temp and get_type(pxl) != pxl_type["burnt"]:
                    #set state to maillard
                    set_pxl_type(pxl,"maillard")

# Cooking the Steak

In [ ]:
def flip(steak):
    heating = steak.pop(-1)
    steak.reverse()
    steak.append(heating)

def plot_heatmaps(image,timestep,interval,total_duration):
    n_plots = total_duration // interval
    i = 0
    arr_of_images = []
    internal_temperature = []
    external_temperature = []
    print("Processing ... ")
    while i < n_plots:
        j = 0
        while j < (interval / timestep):
            duration = round(timestep * j + i * interval,2)

            # ===========================================
            # Flip steak (every 30s)
            # ===========================================
            if duration % 30 == 0:
                flip(image)

            # ===========================================
            # Calculate central pixel temperature
            # ===========================================
            internal_temp = get_temp(image[len(image)//2][len(image[0])//2])
            internal_temperature.append(internal_temp)

            # ===========================================
            # Calculate average outer layer temperature
            # ===========================================
            upper_temp = list(map(lambda x : get_temp(x),image[0]))
            lower_temp = list(map(lambda x : get_temp(x),image[-2]))
            average_upper_temp = sum(upper_temp)/len(upper_temp)
            average_lower_temp = sum(lower_temp)/len(lower_temp)
            external_temp = (average_upper_temp + average_lower_temp)/2
            external_temperature.append(external_temp)
            
            step(image,timestep)            

            j += 1

        arr_of_images.append(copy.deepcopy(image))
        i += 1
    print("Done")
    return (arr_of_images, internal_temperature, external_temperature)

# Visualising the Steak

In [ ]:
def to_color(pxl,min_temp = room_temperature, max_temp = max_temperature):

    temp = get_temp(pxl)

    temp = min_temp if temp < min_temp else max_temp if temp > max_temp else temp
    # if temp < min_temp or temp > max_temp:
    #     print(f"Please enter a temperature between {min_temp} and {max_temp}!")
    heat_index = (temp - min_temp)/(max_temp - min_temp)
    # if get_type(pxl) == "maillard":
    #     color = colors_dict["GOLD"]
    # elif get_type(pxl) == "burnt":
    #     color = colors_dict["BLACK"]
    if get_type(pxl) == "source":
        color = colors_dict["RED"]
    else:
        if get_outer(pxl):
            raw_color = colors_dict["RAW_COLOR"]
            cooked_color = colors_dict["OUTER_COOKED_COLOR"]
        else:
            raw_color = colors_dict["RAW_COLOR"]
            cooked_color = colors_dict["INNER_COOKED_COLOR"]
        color = list(map(lambda x : (((cooked_color[x] - raw_color[x]) * heat_index) + raw_color[x])/255,range(3)))
    return color

def get_color_map(image):
    # image = list(list(get_temp(pixel) for pixel in row) for row in image)
    res = []
    for i in range(len(image)):
        tmp = []
        for j in range(len(image[0])):
            tmp.append(to_color(image[i][j]))
        res.append(tmp)
    return res

In [ ]:
def to_comp_color(pxl):

    if get_type(pxl) == "source":
        color = colors_dict["RED"]
    elif get_type(pxl) == "fat":
        color = colors_dict["FAT"]
    elif get_type(pxl) == "water":
        color = colors_dict["WATER"]
    elif get_type(pxl) == "protein":
        color = colors_dict["RED"]
    return color

def get_comp_color_map(image):
    res = []
    for i in range(len(image)):
        tmp = []
        for j in range(len(image[0])):
            tmp.append(to_comp_color(image[i][j]))
        res.append(tmp)
    return res

# Analysing Data